In [2]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
data = []

links=[]
linkstart = "https://evenementen.uitslagen.nl/2016/marathonrotterdam/uitslag.php?on=1&p=1&tl="

In [3]:
#uitslagen={"2006":74}
#uitslagen={"2007":18}
uitslagen={"2008":70}
#uitslagen={"2009":65}
#uitslagen={"2010":79}
#uitslagen={"2011":74}
#uitslagen={"2012":76}
#uitslagen={"2013":84} 
#uitslagen={"2014":107} 
#uitslagen={"2015":119} 
#uitslagen={"2016":129} 
#uitslagen={"2017":132} 
data = []

links=[]

for i,j in uitslagen.items():
    baseurl="https://evenementen.uitslagen.nl/"+str(i) + "/marathonrotterdam/"
    year = i
    for page in range(1,j+1):
        url= baseurl + "uitslag.php?on=1&p=" + str(page) + "&tl="
        
       
        response = requests.get(url).text
        sleep(randint(1, 3))  # pauze the loop
        soup = BeautifulSoup(response, 'html.parser')
        table = soup.find("table",{"class":"i"})
        links.append(table.findAll('a', href=re.compile('details.php')))
        #column_names = ["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime","Photo","Year"]
        column_names = ["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime","Video","Photo","Year"]

        for row in table.find_all('tr'):
            row_data = []
            for td in row.find_all('td'):
                td_check_img = td.find('img')
                if td_check_img is not None:
                    link = td.img['title']
                    row_data.append(link)
                else:
                    not_link = ''.join(td.stripped_strings)
                    if not_link == '':
                         not_link = None
                    row_data.append(not_link)
            row_data.append(int(year))
            data.append(row_data)   
#df = pd.DataFrame(data[1:], columns=column_names)
df = pd.DataFrame(data[1:])


In [ ]:
#uitslagen={"2009":65}
#uitslagen={"2010":79}
#uitslagen={"2011":74}
#uitslagen={"2012":76}
#uitslagen={"2013":84} 
#uitslagen={"2014":107} 
#uitslagen={"2015":119} 
#uitslagen={"2016":129} 
#uitslagen={"2017":132} 
data = []

links=[]

for i,j in uitslagen.items():
    baseurl="https://evenementen.uitslagen.nl/"+str(i) + "/marathonrotterdam/"
    year = i
    for page in range(1,j+1):
        url= baseurl + "uitslag.php?on=1&p=" + str(page) + "&tl="
        
       
        response = requests.get(url).text
        sleep(randint(1, 3))  # pauze the loop
        soup = BeautifulSoup(response, 'html.parser')
        table = soup.find("table",{"class":"i"})
        links.append(table.findAll('a', href=re.compile('details.php')))
        column_names = ["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime","Photo","Year"]
        #column_names = ["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime","Video","Photo","Year"]

        for row in table.find_all('tr'):
            row_data = []
            for td in row.find_all('td'):
                td_check_img = td.find('img')
                if td_check_img is not None:
                    link = td.img['title']
                    row_data.append(link)
                else:
                    not_link = ''.join(td.stripped_strings)
                    if not_link == '':
                         not_link = None
                    row_data.append(not_link)
            row_data.append(int(year))
            data.append(row_data)   
#df = pd.DataFrame(data[1:], columns=column_names)
df = pd.DataFrame(data[1:])
#column_names = ["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime","Photo","Video","Year"]
column_names = ["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime","Photo","Year"]
df = pd.DataFrame(data[1:], columns=column_names)

#remove the duplicates enries due to empty tr tags
df = df.drop_duplicates()
df.info()
filename = "dataframe" + year + ".csv"
df.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

#flatten the list of the detail links into single list
linklist  = [val for sublist in links for val in sublist]
len(linklist) 

df_link = pd.DataFrame({"links": linklist})
#print(df_link)
filename = "links" + year + ".csv"
df_link.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

results=pd.DataFrame()
start_time = time()
request = 0
for link in linklist:
    url2 = baseurl + str(link.get('href'))
    website_url2 = requests.get(url2).text
    sleep(randint(1, 3))  # pauze the loop
    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request / elapsed_time))
    clear_output(wait=True)
    soup2 = BeautifulSoup(website_url2, "html.parser")
    col = soup2.findAll('td')  # find all td tags
    # mark the TD-cells we want to use

    startnr = col[0].string.replace("Startnummer", "")
    naam = col[4].string
    woonplaats = col[6].string
    afstand = col[8].string
    cat = col[10].string
    totplaats = col[12].string
    catplaats = col[14].string
    snelh = col[16].string
    brutot = col[18].string
    nett = col[20].string

    vijf = col[24].string
    tien = col[26].string
    vtien = col[28].string
    twintig = col[30].string
    half = col[32].string
    vtwintig = col[34].string
    dertig = col[36].string
    vdertig = col[38].string
    veertig = col[40].string

    data2 = {"url": url, "Bib": startnr, "Name": naam, "City": woonplaats, "Distance": afstand, "Agegroup": cat,
            "Place": totplaats, "Agegrouprank": catplaats, "Speed": snelh, "Guntime": brutot, "Chiptime": nett,
            "5KM": vijf, "10KM": tien, "15KM": vtien, "20KM": twintig, "HALF": half, "25KM": vtwintig, "30KM": dertig,
            "35KM": vdertig, "40KM": veertig}  # definieren wat ik wil opslaan in de database
    results = results.append(data2, ignore_index=True)
filename = "results" + year + ".csv"
results.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

uitslagen = pd.merge(results, df, on=["Name", "Guntime", "Agegroup"])
filename = "uitslagen" + year + ".csv"
uitslagen.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

uitslagen.head()
uitslagen.info()

In [4]:
column_names = ["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime","Photo","Year"]
df = pd.DataFrame(data[1:], columns=column_names)
df.head()

,Place,Name,City,Country,AgegroupPlace,Agegroup,Guntime,Photo,Year
0,1,William Kipsang,Kenia,1e,MSR,2:05:49,Video,Foto,2008.0
1,2,Daniel Rono,Kenia,2e,MSR,2:06:58,Video,Foto,2008.0
2,3,Charles Kamathi,Kenia,3e,MSR,2:07:33,Video,Foto,2008.0
3,4,Richard Limo,Kenia,4e,MSR,2:08:43,Video,Foto,2008.0
4,5,Paul Kirui,Kenia,5e,MSR,2:09:46,Video,Foto,2008.0


In [ ]:
#remove the duplicates enries due to empty tr tags
df = df.drop_duplicates()
df.info()

In [ ]:
filename = "dataframe" + year + ".csv"
df.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [ ]:
#flatten the list of the detail links into single list
linklist  = [val for sublist in links for val in sublist]
len(linklist)           

In [ ]:
df_link = pd.DataFrame({"links": linklist})
#print(df_link)
filename = "links" + year + ".csv"
df_link.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [ ]:
results=pd.DataFrame()
start_time = time()
request = 0
for link in linklist:
    url2 = baseurl + str(link.get('href'))
    website_url2 = requests.get(url2).text
    sleep(randint(1, 3))  # pauze the loop
    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request / elapsed_time))
    clear_output(wait=True)
    soup2 = BeautifulSoup(website_url2, "html.parser")
    col = soup2.findAll('td')  # find all td tags
    # mark the TD-cells we want to use

    startnr = col[0].string.replace("Startnummer", "")
    naam = col[4].string
    woonplaats = col[6].string
    afstand = col[8].string
    cat = col[10].string
    totplaats = col[12].string
    catplaats = col[14].string
    snelh = col[16].string
    brutot = col[18].string
    nett = col[20].string

    vijf = col[24].string
    tien = col[26].string
    vtien = col[28].string
    twintig = col[30].string
    half = col[32].string
    vtwintig = col[34].string
    dertig = col[36].string
    vdertig = col[38].string
    veertig = col[40].string

    data2 = {"url": url, "Bib": startnr, "Name": naam, "City": woonplaats, "Distance": afstand, "Agegroup": cat,
            "Place": totplaats, "Agegrouprank": catplaats, "Speed": snelh, "Guntime": brutot, "Chiptime": nett,
            "5KM": vijf, "10KM": tien, "15KM": vtien, "20KM": twintig, "HALF": half, "25KM": vtwintig, "30KM": dertig,
            "35KM": vdertig, "40KM": veertig}  # definieren wat ik wil opslaan in de database
    results = results.append(data2, ignore_index=True)
filename = "results" + year + ".csv"
results.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [ ]:
uitslagen = pd.merge(results, df, on=["Name", "Guntime", "Agegroup"])
filename = "uitslagen" + year + ".csv"
uitslagen.to_csv (filename, index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [ ]:
    result_test =pd.DataFrame()
    url2 = "https://evenementen.uitslagen.nl/2014/marathonrotterdam/details.php?s=1&o=1&t=nl"
    website_url2 = requests.get(url2).text
    sleep(randint(1, 3))  # pauzeer de loop
    # Monitor the requests
    soup2 = BeautifulSoup(website_url2, "html.parser")
    col = soup2.findAll('td')  # alle td tags vinden
    #print(col)
    # aangeven welke td-cellen we willen hebben

    startnr = col[0].string.replace("Startnummer", "")
    naam = col[4].string
    woonplaats = col[6].string
    afstand = col[8].string
    cat = col[10].string
    totplaats = col[12].string
    catplaats = col[14].string
    snelh = col[16].string
    brutot = col[18].string
    nett = col[20].string

    vijf = col[24].string
    tien = col[26].string
    vtien = col[28].string
    twintig = col[30].string
    half = col[32].string
    vtwintig = col[34].string
    dertig = col[36].string
    vdertig = col[38].string
    veertig = col[40].string

    data2 = {"url": url, "Bib": startnr, "Name": naam, "City": woonplaats, "Distance": afstand, "Agegroup": cat,
            "Rank": totplaats, "Agegrouprank": catplaats, "Speed": snelh, "Guntime": brutot, "Chiptime": nett,
            "5KM": vijf, "10KM": tien, "15KM": vtien, "20KM": twintig, "HALF": half, "25KM": vtwintig, "30KM": dertig,
            "35KM": vdertig, "40KM": veertig}  # definieren wat ik wil opslaan in de database
    result_test = result_test.append(data2, ignore_index=True)

In [ ]:
result_test.head()

# 2017

In [ ]:
results=pd.DataFrame()
start_time = time()
request = 0
for link in linklist:
    url2 = baseurl + str(link.get('href'))
    website_url2 = requests.get(url2).text
    sleep(randint(1, 3))  # pauze the loop
    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request / elapsed_time))
    clear_output(wait=True)
    soup2 = BeautifulSoup(website_url2, "html.parser")
    col = soup2.findAll('td')  # find all td tags
    # mark the TD-cells we want to use

    startnr = col[0].string.replace("Startnummer", "")
    naam = col[4].string
    woonplaats = col[6].string
    afstand = col[8].string
    cat = col[10].string
    totplaats = col[12].string
    catplaats = col[14].string
    snelh = col[16].string
    brutot = col[18].string
    nett = col[20].string

    vijf = col[23].string
    tien = col[25].string
    vtien = col[27].string
    twintig = col[29].string
    half = col[31].string
    vtwintig = col[33].string
    dertig = col[35].string
    vdertig = col[37].string
    veertig = col[39].string

    data2 = {"url": url, "Bib": startnr, "Name": naam, "City": woonplaats, "Distance": afstand, "Agegroup": cat,
            "Place": totplaats, "Agegrouprank": catplaats, "Speed": snelh, "Guntime": brutot, "Chiptime": nett,
            "5KM": vijf, "10KM": tien, "15KM": vtien, "20KM": twintig, "HALF": half, "25KM": vtwintig, "30KM": dertig,
            "35KM": vdertig, "40KM": veertig}  # definieren wat ik wil opslaan in de database
    results = results.append(data2, ignore_index=True)
export_csv = results.to_csv (r'C:\Users\Ivo\Desktop\export_results2017.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [ ]:
uitslagen.head()
uitslagen.info()


In [ ]:
df_2013 = pd.read_csv('uitslagen2013.csv')
df_2014 = pd.read_csv('uitslagen2014.csv')
df_2015 = pd.read_csv('uitslagen2015.csv')
df_2016 = pd.read_csv('uitslagen2016.csv')
df_2017 = pd.read_csv('uitslagen2017.csv')
jaren = [df_2013, df_2014, df_2015, df_2016, df_2017]
uitslag = pd.concat(jaren, sort=False)
uitslag[uitslag["Place_y"]==1]